# System Administrators
## Setup : this should seem familiar

In [1]:
from dotenv import load_dotenv
from os import getenv
from faker import Faker
fake = Faker()
import random

if load_dotenv():
    key, secret, url = getenv('qkey'), getenv('qsecret'), getenv('qurl')
    try:
        assert key, "Key not loaded"
        assert secret, "Secret not loaded"
        assert url, "Url not loaded"
        print("All environment variables loaded successfully")
    except AssertionError as e:
        print(e)
        print("Ensure your .env file has url, key and secret")
else:
    print('Make sure you have a .env file')

All environment variables loaded successfully


In [2]:
from bbrest import BbRest
bb = BbRest(key, secret, url)

# Three Use Cases:

* 1. Creating fake users, three fake courses, and fake memberships. 
* 2. Getting a users last accessed courses 
* 3. Disabling and enabling a course and user. 

## Creating Fake Users, Courses and Memberships

### Creating Fake Courses
We are going to create 3 fake courses.  Since we are only creating a handful, we will enter the information manually.

In [3]:
r = bb.CreateCourse(payload={'name':'DevCon-Demo1','courseId':'NAC-DevCon1'})
print(r.json())

{'id': '_332335_1', 'uuid': 'ff72a5d667a149a299e4c4611355d665', 'externalId': 'NAC-DevCon1', 'dataSourceId': '_2_1', 'courseId': 'NAC-DevCon1', 'name': 'DevCon-Demo1', 'created': '2019-07-22T02:59:28.000Z', 'organization': False, 'ultraStatus': 'Classic', 'allowGuests': False, 'readOnly': False, 'availability': {'available': 'Yes', 'duration': {'type': 'Continuous'}}, 'enrollment': {'type': 'InstructorLed'}, 'locale': {'force': False}, 'externalAccessUrl': 'https://bbqa.cc.ku.edu/webapps/blackboard/execute/courseMain?course_id=_332335_1&sc='}


In [4]:
r1 = bb.CreateCourse(payload={'name':'DevCon-Demo2','courseId':'NAC-DevCon2'})
r2 = bb.CreateCourse(payload={'name':'DevCon-Demo3','courseId':'NAC-DevCon3'})

print(r1.status_code)
print(r2.status_code)

201
201


### Creating Fake Users
For this, we are going to create 50 users with the help from the libary 'faker'.  Faker generates random names / addresses / passwords ... basically all user info needed for this use case.

In [5]:
students = []
for _ in range(50):
    name = fake.name()
    while len(name.split()) > 2:
        name = fake.name()
    names = name.split()  
    first = names[0]
    last = names[-1]
    userName = f'{first[0]}{last}{fake.random_number(digits=3)}'.lower()
    studentId = fake.random_number(digits=9,fix_len=True)
    email = f'{userName}@ku.edu'
    password = fake.password()
    
    student = {'name':{'family': last, 'given': first},
               'userName': userName,
               'password': password,
               'studentId': studentId,
               'externalId': studentId,
               'contact': {'email':email}
              }
    
    
    r = bb.CreateUser(payload=student)
    
    if r.status_code == 201:
        print(f'Created account for {name}')
        students.append(student)
    else:
        print(r.text)
        print(f'Account not created for {name}')

Created account for Omar Branch
Created account for Michelle Mitchell
Created account for Edward Garcia
Created account for Jessica Serrano
Created account for John Morris
Created account for Laurie Curry
Created account for Kristina Ruiz
Created account for Paul Colon
Created account for Shawn Harris
Created account for William Holmes
Created account for Tammy Miller
Created account for Marisa Hunter
Created account for Stacy Perkins
Created account for Steven Ruiz
Created account for Molly Henson
Created account for Anna Wilson
Created account for Megan Henderson
Created account for Sarah Obrien
Created account for Brian Long
Created account for James Harris
Created account for Ryan Alexander
Created account for Brandon Williams
{"status":409,"message":"Unique ID conflicts with existing record: userName"}
Account not created for Patrick Smith
Created account for William Hall
Created account for Amber Lang
Created account for Tracy Lewis
Created account for Claudia Schneider
Created a

### Creating Fake Memberships
We are going to add my account as an instructor in these three courses, and the students each into one of the three courses.

In [6]:
courses = ['NAC-DevCon1', 'NAC-DevCon2', 'NAC-DevCon3']
for course in courses:
    r = bb.CreateMembership(userId='m500d520', courseId=course, payload={'courseRoleId':'Instructor'})
    print(r.status_code)

201
201
201


In [7]:
import random

'NAC-DevCon3'

In [11]:
random.choice(courses)

'NAC-DevCon3'

In [12]:
for student in students:
    userName = student['userName']
    course = random.choice(courses)
    r = bb.CreateMembership(userId=userName, courseId=course, payload={})
    if r.status_code == 201:
        print(f'Added {userName} to {course}')
    else:
        print(r.text)

Added obranch826 to NAC-DevCon1
Added mmitchell43 to NAC-DevCon3
Added egarcia770 to NAC-DevCon2
Added jserrano635 to NAC-DevCon3
Added jmorris691 to NAC-DevCon1
Added lcurry799 to NAC-DevCon1
Added kruiz303 to NAC-DevCon3
Added pcolon57 to NAC-DevCon2
Added sharris993 to NAC-DevCon2
Added wholmes200 to NAC-DevCon2
Added tmiller415 to NAC-DevCon1
Added mhunter962 to NAC-DevCon3
Added sperkins450 to NAC-DevCon1
Added sruiz294 to NAC-DevCon2
Added mhenson8 to NAC-DevCon2
Added awilson488 to NAC-DevCon1
Added mhenderson650 to NAC-DevCon1
Added sobrien739 to NAC-DevCon3
Added blong612 to NAC-DevCon3
Added jharris806 to NAC-DevCon3
Added ralexander581 to NAC-DevCon3
Added bwilliams179 to NAC-DevCon3
Added whall618 to NAC-DevCon1
Added alang463 to NAC-DevCon1
Added tlewis964 to NAC-DevCon1
Added cschneider623 to NAC-DevCon2
Added lbooth158 to NAC-DevCon1
Added knorris297 to NAC-DevCon2
Added jtaylor640 to NAC-DevCon1
Added adean382 to NAC-DevCon1
Added ahill873 to NAC-DevCon3
Added shodges59

## 2. Getting a User's Last accessed courses
This is the start of some useful reporting, and can help focuse the search space for grade or access data.

In [13]:
r = bb.GetUserMemberships('m500d520', 
                      params={'lastAccessed':'2019-07-01',
                              'lastAccessedCompare':'greaterOrEqual'})
r.json()

{'results': [{'userId': '_702000_1',
   'courseId': '_332335_1',
   'dataSourceId': '_2_1',
   'created': '2019-07-22T03:00:30.000Z',
   'availability': {'available': 'Yes'},
   'courseRoleId': 'Instructor',
   'lastAccessed': '2019-07-22T03:02:34.000Z'},
  {'userId': '_702000_1',
   'courseId': '_332336_1',
   'dataSourceId': '_2_1',
   'created': '2019-07-22T03:00:31.000Z',
   'availability': {'available': 'Yes'},
   'courseRoleId': 'Instructor',
   'lastAccessed': '2019-07-22T03:02:34.000Z'},
  {'userId': '_702000_1',
   'courseId': '_332337_1',
   'dataSourceId': '_2_1',
   'created': '2019-07-22T03:00:31.000Z',
   'availability': {'available': 'Yes'},
   'courseRoleId': 'Instructor',
   'lastAccessed': '2019-07-22T03:02:34.000Z'}],
 'paging': {'nextPage': '/learn/api/public/v1/users/userName:m500d520/courses?lastAccessedCompare=greaterOrEqual&lastAccessed=2019-07-01&offset=3'}}

In [14]:
courses = r.json()['results']
for course in courses:
    courseId = course['courseId']
    r = bb.GetCourse(courseId)
    course_info = r.json()
    print(course_info['name'])

DevCon-Demo1
DevCon-Demo2
DevCon-Demo3


## 3. Disabling and enabling a user and a course
Sometimes SIS integrations cause issues. We use SAIP, and courses have been disabled when a section changes and enrollments have been disabled when a student changes sections.  Here are quick ways to correct this with the API.

In [15]:
#Causing course and user to be disabled on purpose
r1 = bb.UpdateCourse('NAC-DevCon2', payload={'availability':{'available':'Disabled'}})
r2 = bb.UpdateMembership(userId='m500d520', courseId='NAC-DevCon1', payload={'availability':{'available':'Disabled'}})

print(r1.json())
print(r2.json())

{'id': '_332336_1', 'uuid': '1fadde2f4af64d2584fddc27c63b153b', 'externalId': 'NAC-DevCon2', 'dataSourceId': '_2_1', 'courseId': 'NAC-DevCon2', 'name': 'DevCon-Demo2', 'created': '2019-07-22T02:59:33.000Z', 'organization': False, 'ultraStatus': 'Classic', 'allowGuests': False, 'readOnly': False, 'availability': {'available': 'Disabled', 'duration': {'type': 'Continuous'}}, 'enrollment': {'type': 'InstructorLed'}, 'locale': {'force': False}}
{'userId': '_702000_1', 'courseId': '_332335_1', 'dataSourceId': '_2_1', 'created': '2019-07-22T03:00:30.000Z', 'availability': {'available': 'Disabled'}, 'courseRoleId': 'Instructor', 'lastAccessed': '2019-07-22T03:03:34.000Z'}


In [19]:
r1 = bb.UpdateCourse('NAC-DevCon2', payload={'availability':{'available':'Yes'}})
r2 = bb.UpdateMembership(userId='m500d520', courseId='NAC-DevCon1', payload={'availability':{'available':'Yes'}})
r3 = bb.UpdateMembership(userId='m500d520', courseId='NAC-DevCon2', payload={'availability':{'available':'Yes'}})

print(r1.json())
print(r2.json())
print(r3.json())

{'id': '_332336_1', 'uuid': '1fadde2f4af64d2584fddc27c63b153b', 'externalId': 'NAC-DevCon2', 'dataSourceId': '_2_1', 'courseId': 'NAC-DevCon2', 'name': 'DevCon-Demo2', 'created': '2019-07-22T02:59:33.000Z', 'organization': False, 'ultraStatus': 'Classic', 'allowGuests': False, 'readOnly': False, 'availability': {'available': 'Yes', 'duration': {'type': 'Continuous'}}, 'enrollment': {'type': 'InstructorLed'}, 'locale': {'force': False}, 'externalAccessUrl': 'https://bbqa.cc.ku.edu/webapps/blackboard/execute/courseMain?course_id=_332336_1&sc='}
{'userId': '_702000_1', 'courseId': '_332335_1', 'dataSourceId': '_2_1', 'created': '2019-07-22T03:00:30.000Z', 'availability': {'available': 'Yes'}, 'courseRoleId': 'Instructor', 'lastAccessed': '2019-07-22T03:05:34.000Z'}
{'userId': '_702000_1', 'courseId': '_332336_1', 'dataSourceId': '_2_1', 'created': '2019-07-22T03:00:31.000Z', 'availability': {'available': 'Yes'}, 'courseRoleId': 'Instructor', 'lastAccessed': '2019-07-22T03:05:34.000Z'}


# Clean-up
Run this after the other demonstrations.  You need the users, courses and memberships for the next two Notebooks. 

In [13]:
for user in [student['externalId'] for student in students]:
    r = bb.DeleteUser(f'externalId:{user}')
    print(r.status_code)

204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204
204


In [13]:
for course in ['NAC-DevCon1', 'NAC-DevCon2', 'NAC-DevCon3']:
    r = bb.DeleteMembership(courseId=course, userId='m500d520')
    r = bb.DeleteCourse(course)
    print(r.status_code)

204
204
204


In [11]:
students = bb.GetUsers(params={'created':'2019-07-20',
                              'createdCompare':'greaterOrEqual'}).json()['results']

In [12]:
len(students)

49